In [13]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt
import glob
import re

In [2]:
# Set up paths

path_to_data = '/expanse/nfs/cw3e/cwp140/'     # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [26]:
yr = 1987
fname = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_{1}.nc'.format('SNOW', yr)
ds = xr.open_dataset(fname)
ds

<xarray.Dataset>
Dimensions:  (time: 364, y: 250, x: 320)
Coordinates:
  * time     (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    lat      (y, x) float32 ...
    lon      (y, x) float32 ...
Dimensions without coordinates: y, x
Data variables:
    SNOW     (time, y, x) float32 ...
Attributes:
    contact:  rtladerjr@alaska.edu
    info:     Alaska CASC
    data:     Downscaled CFSR
    format:   version 1
    date:     Wed Jan  9 04:37:35 AKST 2019

In [24]:
fname_pattern = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format('SNOW')
fname_lst = sorted(glob.glob(fname_pattern))

yr_lst = np.arange(1980, 2020, 1)
ds_lst = []
for i, yr in enumerate(yr_lst):
    print(yr)
    fname = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_{1}.nc'.format('SNOW', yr)
    ds = xr.open_dataset(fname)
    time_lst = pd.date_range('{0}-01-01'.format(yr), '{0}-12-31'.format(yr), freq='1D')
    time_lst = time_lst[~((time_lst.month == 2) & (time_lst.day == 29))]
    ds['time'] = time_lst
    ds_lst.append(ds)
ds_lst[0]

1980
1981
1982
1983
1984
1985
1986
1987


ValueError: conflicting sizes for dimension 'time': length 365 on 'time' and length 364 on {'time': 'SNOW', 'y': 'SNOW', 'x': 'SNOW'}

In [3]:
temporal_res = 'daily'
community_lst = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']
varname = 'SR' # 'PCPT' or 'UV' or 'SR'

if varname == 'SR':
    ## open SWE
    fname_pattern = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format('SNOW')
    SNOW = xr.open_mfdataset(fname_pattern, combine='by_coords', engine='netcdf4')

    ## open PCPT
    fname_pattern = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format('PCPT')
    PCPT = xr.open_mfdataset(fname_pattern, combine='by_coords', engine='netcdf4')
    PCPT = PCPT.resample(time="1D").mean('time') # resample WRF data to be m s-1

else:

    fname_pattern = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format(varname)
    wrf = xr.open_mfdataset(fname_pattern, combine='by_coords', engine='netcdf4')
    if temporal_res == '6hr':
        wrf = wrf.resample(time="6H").sum('time') # resample WRF data to be mm per day
    elif (temporal_res == 'daily') & (varname == 'PCPT'):
        wrf = wrf.resample(time="1D").sum('time') # resample WRF data to be mm per day
        
    elif (temporal_res == 'daily') & (varname == 'UV'):
        wrf = wrf.sel(lev='1000')
        wrf = wrf.resample(time="1D").mean('time') # resample WRF data to be m s-1


ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed


TypeError: Cannot combine along dimension 'time' with mixed types. Found: int64, Timestamp.

In [ ]:
### calculate percentile for fraction of rain to snow
## SNOW is snow water equivalent
## PCPT is total precipitation
## fraction of frozen precipitation (SR) = SWE/PCPT
SNOW

In [ ]:
PCPT

In [ ]:
%%time
## need to rechunk so time is a single chunk
wrf = wrf.chunk(dict(time=-1))

quantile_arr = np.array([0, .75, .95])

## Calculate the percentiles
quantile = wrf.quantile(quantile_arr, dim=['time'], skipna=True)

In [ ]:
quantile

In [ ]:
%%time
# write to netCDF
fname = os.path.join(path_to_data, 'preprocessed/SEAK-WRF-PCPT/PCPT_quantiles_{0}.nc'.format(temporal_res))
quantile.load().to_netcdf(path=fname, mode = 'w', format='NETCDF4')